# PP2A - Single gene analysis in GP2 data

## Description

Using "individual level data"

### 0. Getting Started

- Loading Python libraries
- Defining functions
- Installing packages

### 1. Copy data from workspace to cloud environment

### 2. Extract PP2A

### 3. Annotate PP2A variants

### 4. Extract coding/non-syn variants

### 5. Calculate frequency in cases versus controls

### 6. Calculate frequency (homozygotes) in cases versus controls

### 7. Save out results

## Getting Started

### Loading Python libraries

### Defining functions

In [67]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for DatAACame functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

In [68]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DatAACame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Set paths

In [69]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')


## GP2 v3.0
## Explicitly define release v3.0 path 
GP2_RELEASE_PATH = 'gs://gp2tier2/release3_31102022'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v3.0')
print(f'Path to GP2 v3.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v3.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v3.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')


## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/2022_v3release_1115'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

Billing and Workspace
Workspace Name: PP2A
Billing Project: terra-02e7cd23
Workspace Bucket, where you can upload and download data: gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0

GP2 v3.0
Path to GP2 v3.0 Clinical Data: gs://gp2tier2/release3_31102022/clinical_data
Path to GP2 v3.0 Raw Genotype Data: gs://gp2tier2/release3_31102022/raw_genotypes
Path to GP2 v3.0 Imputed Genotype Data: gs://gp2tier2/release3_31102022/imputed_genotypes
AMP-PD v3.0
Path to AMP-PD v3.0 Clinical Data: gs://amp-pd-data/releases/2022_v3release_1115/clinical
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink/pfiles



### Install packages

#### Install Plink 1.9 and Plink 2.0

In [70]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/


In [71]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Remote restrictions

In [72]:
%%bash

# chmod plink 1.9 
chmod u+x /home/jupyter/tools/plink

In [73]:
%%bash

# chmod plink 2.0
chmod u+x /home/jupyter/tools/plink2

### Install ANNOVAR

In [74]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [75]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Install ANNOVAR: Download sources of annotation 

In [76]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

In [77]:
%%bash
ls /home/jupyter/tools/annovar/

annotate_variation.pl
coding_change.pl
convert2annovar.pl
example
humandb
retrieve_seq_from_fasta.pl
table_annovar.pl
variants_reduction.pl


## AAC

## Copy data from GP2 bucket to workspace

In [78]:
# Make a directory
print("Making a working directory")
WORK_DIR = f'/home/jupyter/PP2A_GP2/'
shell_do(f'mkdir -p {WORK_DIR}') # f' significa f-string - contiene expresiones para ejecutar el codigo

Making a working directory


Executing: mkdir -p /home/jupyter/PP2A_GP2/


In [79]:
# Check directory where GP2 data is
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}')

Executing: gsutil -u terra-02e7cd23 ls gs://gp2tier2/release3_31102022/imputed_genotypes


gs://gp2tier2/release3_31102022/imputed_genotypes/AAC/
gs://gp2tier2/release3_31102022/imputed_genotypes/AFR/
gs://gp2tier2/release3_31102022/imputed_genotypes/AJ/
gs://gp2tier2/release3_31102022/imputed_genotypes/AMR/
gs://gp2tier2/release3_31102022/imputed_genotypes/CAS/
gs://gp2tier2/release3_31102022/imputed_genotypes/EAS/
gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/
gs://gp2tier2/release3_31102022/imputed_genotypes/MDE/
gs://gp2tier2/release3_31102022/imputed_genotypes/SAS/


In [80]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/AAC/chr9* {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 -m cp -r gs://gp2tier2/release3_31102022/imputed_genotypes/AAC/chr9* /home/jupyter/PP2A_GP2/


Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AAC/chr9_AAC_release3.log...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AAC/chr9_AAC_release3.psam...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AAC/chr9_AAC_release3.pgen...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AAC/chr9_AAC_release3.pvar...
/ [4/4 files][535.9 MiB/535.9 MiB] 100% Done                                    
Operation completed over 4 objects/535.9 MiB.                                    


### Create a covariate file with GP2 data

In [81]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_CLINICAL_RELEASE_PATH}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_CLINICAL_RELEASE_PATH}/master_key_release3_final.csv {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 ls gs://gp2tier2/release3_31102022/clinical_data


gs://gp2tier2/release3_31102022/clinical_data/
gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv
gs://gp2tier2/release3_31102022/clinical_data/release3_31102022_data_dictionary.csv


Executing: gsutil -u terra-02e7cd23 -m cp -r gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv /home/jupyter/PP2A_GP2/


Copying gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv...
/ [1/1 files][  1.6 MiB/  1.6 MiB] 100% Done                                    
Operation completed over 1 objects/1.6 MiB.                                      


In [82]:
cov = pd.read_csv(f'{WORK_DIR}/master_key_release3_final.csv')
cov.columns

Index(['GP2ID', 'GP2sampleID', 'manifest_id', 'phenotype', 'pheno_for_qc',
       'other_pheno', 'sex_for_qc', 'age', 'age_of_onset', 'age_at_diagnosis',
       'age_at_death', 'race_for_qc', 'family_history_for_qc', 'region_for_qc',
       'study', 'pruned', 'pruned_reason', 'label', 'related'],
      dtype='object')

In [83]:
cov_reduced = cov[['GP2sampleID','GP2sampleID','sex_for_qc', 'age', 'phenotype']]
cov_reduced.columns = ['FID','IID', 'SEX','AGE','PHENO']
cov_reduced

,FID,IID,SEX,AGE,PHENO
0,SYNAPS-KZ_000001_s1,SYNAPS-KZ_000001_s1,2,64.0,Control
1,SYNAPS-KZ_000002_s1,SYNAPS-KZ_000002_s1,1,53.0,Control
2,SYNAPS-KZ_000003_s1,SYNAPS-KZ_000003_s1,2,59.0,Control
3,SYNAPS-KZ_000004_s1,SYNAPS-KZ_000004_s1,1,57.0,Control
4,SYNAPS-KZ_000005_s1,SYNAPS-KZ_000005_s1,1,59.0,Control
...,...,...,...,...,...
16852,PPMI_000789_s1,PPMI_000789_s1,2,NaN,Other
16853,PPMI_000781_s1,PPMI_000781_s1,1,57.0,Other
16854,PPMI_000818_s1,PPMI_000818_s1,1,60.0,Other
16855,PPMI_000841_s1,PPMI_000841_s1,2,68.0,Other


In [84]:
conditions = [
    (cov_reduced['PHENO'] == "Case"),
    (cov_reduced['PHENO'] == "Control")]

In [85]:
choices = [2,1]
cov_reduced['PHENOTYPE'] = np.select(conditions, choices, default=-9).astype(np.int64)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [86]:
cov_reduced.reset_index(inplace=True)
cov_reduced.drop(columns=["index"], inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [87]:
cov_reduced.drop(columns=['PHENO'], inplace=True)

In [88]:
sex = cov_reduced[['FID','IID','SEX']]
sex.to_csv(f'{WORK_DIR}/SEX.txt',index=False, sep="\t")

In [89]:
pheno = cov_reduced[['FID','IID','PHENOTYPE']]
pheno.to_csv(f'{WORK_DIR}/PHENO.txt',index=False, sep="\t")

## Extract PP2A

In [90]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

# PODXL: gene positions on hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/tools/plink2 \
--pfile chr9_AAC_release3 \
--chr 9 \
--from-bp 129110950  \
--to-bp 129148946 \
--make-bed \
--out pheno_PP2A_AAC

PLINK v2.00a3.3LM AVX2 Intel (3 Jun 2022)      www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PP2A_AAC.log.
Options in effect:
  --chr 9
  --from-bp 129110950
  --make-bed
  --out pheno_PP2A_AAC
  --pfile chr9_AAC_release3
  --to-bp 129148946

Start time: Thu Jun  1 15:44:57 2023
14998 MiB RAM detected; reserving 7499 MiB for main workspace.
Using up to 4 compute threads.
1216 samples (742 females, 474 males; 1216 founders) loaded from
chr9_AAC_release3.psam.
871030 variants loaded from chr9_AAC_release3.pvar.
1 binary phenotype loaded (150 cases, 1065 controls).
310 variants remaining after main filters.
Writing pheno_PP2A_AAC.fam ... done.
Writing pheno_PP2A_AAC.bim ... done.
Writing pheno_PP2A_AAC.bed ... 0%done.
End time: Thu Jun  1 15:44:58 2023


### Visualize plink files bim, fam and bed

In [91]:
%%bash

# Visualize bim file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_AAC.bim

9	chr9:129110977:A:C	0	129110977	C	A
9	chr9:129111023:G:C	0	129111023	C	G
9	chr9:129111059:G:A	0	129111059	A	G
9	chr9:129111157:A:T	0	129111157	T	A
9	chr9:129111181:T:C	0	129111181	C	T
9	chr9:129111184:G:C	0	129111184	C	G
9	chr9:129111359:T:G	0	129111359	G	T
9	chr9:129111392:G:C	0	129111392	C	G
9	chr9:129111606:T:C	0	129111606	C	T
9	chr9:129111885:C:T	0	129111885	T	C


In [92]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_AAC.fam

0	BCM_000096_s1	0	0	1	2
0	BCM_000142_s1	0	0	1	2
0	BCM_000150_s1	0	0	2	2
0	BCM_000173_s1	0	0	1	2
0	BCM_000204_s1	0	0	2	2
0	BCM_000215_s1	0	0	1	2
0	BCM_000230_s1	0	0	1	2
0	BCM_000270_s1	0	0	2	1
0	BCM_000277_s1	0	0	2	1
0	BCM_000286_s1	0	0	1	2


In [93]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_AAC.bed

l������������������������������������?����������������������3��??����������/������������������������������������������������������������������?�������������������������>>���������������������������������������:���������������������������������������������������ﾾ��믿�����������������﾿�����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������쿿����������������������������������������������������������������������������������������������������������>������������������������������������������������ο��������?�����������~�����������������������������������>��������������ﹿ���������������������������������������������������������쿿��������������������������������������������������������

In [94]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

# Turn binary files into VCF
/home/jupyter/tools/plink \
--bfile pheno_PP2A_AAC \
--recode vcf-fid \
--out pheno_PP2A_AAC

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PP2A_AAC.log.
Options in effect:
  --bfile pheno_PP2A_AAC
  --out pheno_PP2A_AAC
  --recode vcf-fid

14998 MB RAM detected; reserving 7499 MB for main workspace.
310 variants loaded from .bim file.
1216 people (474 males, 742 females) loaded from .fam.
1215 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1216 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62%63%64%65

## Annotate PP2A variants

In [95]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR
export PATH=$PATH:/home/jupyter/tools/rvtests/third/tabix-0.2.6/

### Bgzip and Tabix
bgzip pheno_PP2A_AAC.vcf

tabix -f -p vcf pheno_PP2A_AAC.vcf.gz
tabix -f -p vcf pheno_PP2A_AAC.vcf.gz

[bgzip] can't create pheno_PP2A_AAC.vcf.gz: File exists


In [96]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

### Annotate variants using ANNOVAR: https://annovar.openbioinformatics.org/en/latest/ 
perl /home/jupyter/tools/annovar/table_annovar.pl pheno_PP2A_AAC.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out pheno_PP2A_AAC.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 pheno_PP2A_AAC.vcf.gz > pheno_PP2A_AAC.annovar.avinput>
NOTICE: Finished reading 317 lines from VCF file
NOTICE: A total of 310 locus in VCF file passed QC threshold, representing 285 SNPs (213 transitions and 72 transversions) and 25 indels/substitutions
NOTICE: Finished writing allele frequencies based on 346560 SNP genotypes (259008 transitions and 87552 transversions) and 30400 indels/substitutions for 1216 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl pheno_PP2A_AAC.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile pheno_PP2A_AAC.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.pl -genean

## Extract coding and non-syn variants

In [97]:
# Visualize multianno file
PP2A_AAC = pd.read_csv(f'{WORK_DIR}/pheno_PP2A_AAC.annovar.hg38_multianno.txt', sep = '\t')
PP2A_AAC.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo1219,Otherinfo1220,Otherinfo1221,Otherinfo1222,Otherinfo1223,Otherinfo1224,Otherinfo1225,Otherinfo1226,Otherinfo1227,Otherinfo1228
0,9,129110977,129110977,A,C,UTR5,PTPA,NM_021131:c.-624A>C,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/0
1,9,129111023,129111023,G,C,UTR5,PTPA,NM_021131:c.-578G>C,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2,9,129111059,129111059,G,A,intronic,PTPA,.,.,.,...,0/1,0/0,./.,0/1,0/0,0/1,0/0,0/0,0/0,0/1
3,9,129111157,129111157,A,T,intronic,PTPA,.,.,.,...,0/1,0/0,./.,0/1,0/0,0/1,0/0,0/0,0/0,0/1
4,9,129111181,129111181,T,C,intronic,PTPA,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [98]:
PP2A_AAC.count()

Chr              310
Start            310
End              310
Ref              310
Alt              310
                ... 
Otherinfo1224    310
Otherinfo1225    310
Otherinfo1226    310
Otherinfo1227    310
Otherinfo1228    310
Length: 1239, dtype: int64

In [99]:
# Filter exonic variants
coding_AAC = PP2A_AAC[PP2A_AAC['Func.refGene'] == 'exonic']
coding_AAC.count()
coding_AAC.head(5)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo1219,Otherinfo1220,Otherinfo1221,Otherinfo1222,Otherinfo1223,Otherinfo1224,Otherinfo1225,Otherinfo1226,Otherinfo1227,Otherinfo1228
8,9,129111606,129111606,T,C,exonic,PTPA,.,synonymous SNV,"PTPA:NM_001271832:exon1:c.T6C:p.A2A,PTPA:NM_17...",...,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/0
94,9,129123135,129123135,C,T,exonic,PTPA,.,synonymous SNV,"PTPA:NM_001193397:exon3:c.C108T:p.S36S,PTPA:NM...",...,0/1,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
183,9,129134882,129134882,A,C,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.A461C:p.K154T,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
199,9,129136522,129136522,C,T,exonic,PTPA,.,synonymous SNV,"PTPA:NM_001271832:exon6:c.C525T:p.A175A,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
293,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [100]:
# Filter exonic and non-syn vars
coding_nonsynonymous_AAC = PP2A_AAC[(PP2A_AAC['Func.refGene'] == 'exonic') & (PP2A_AAC['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous_AAC.count(1)

183    1239
293    1239
dtype: int64

In [101]:
coding_nonsynonymous_AAC.head(1)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo1219,Otherinfo1220,Otherinfo1221,Otherinfo1222,Otherinfo1223,Otherinfo1224,Otherinfo1225,Otherinfo1226,Otherinfo1227,Otherinfo1228
183,9,129134882,129134882,A,C,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.A461C:p.K154T,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [102]:
print(coding_nonsynonymous_AAC)

     Chr      Start        End Ref Alt Func.refGene Gene.refGene  \
183    9  129134882  129134882   A   C       exonic         PTPA   
293    9  129147457  129147457   C   T       exonic         PTPA   

    GeneDetail.refGene ExonicFunc.refGene  \
183                  .  nonsynonymous SNV   
293                  .  nonsynonymous SNV   

                                      AAChange.refGene  ... Otherinfo1219  \
183  PTPA:NM_001271832:exon5:c.A461C:p.K154T,PTPA:N...  ...           0/0   
293  PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...  ...           0/0   

     Otherinfo1220 Otherinfo1221 Otherinfo1222  Otherinfo1223  Otherinfo1224  \
183            0/0           0/0           0/0            0/0            0/0   
293            0/0           0/0           0/0            0/0            0/0   

    Otherinfo1225 Otherinfo1226 Otherinfo1227 Otherinfo1228  
183           0/0           0/0           0/0           0/0  
293           0/0           0/0           0/0           0/0  


In [103]:
coding_nonsynonymous_AAC.to_csv(f'{WORK_DIR}/coding_nonsynonymous.txt', sep = '\t', index = False)
coding_nonsynonymous_AAC.head(9)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo1219,Otherinfo1220,Otherinfo1221,Otherinfo1222,Otherinfo1223,Otherinfo1224,Otherinfo1225,Otherinfo1226,Otherinfo1227,Otherinfo1228
183,9,129134882,129134882,A,C,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.A461C:p.K154T,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
293,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [104]:
# Filter intronic variants
intronic_AAC = PP2A_AAC[PP2A_AAC['Func.refGene'] == 'intronic']
intronic_AAC.count()

Chr              285
Start            285
End              285
Ref              285
Alt              285
                ... 
Otherinfo1224    285
Otherinfo1225    285
Otherinfo1226    285
Otherinfo1227    285
Otherinfo1228    285
Length: 1239, dtype: int64

In [105]:
# Filter UTR3 variants
UTR3_AAC = PP2A_AAC[PP2A_AAC['Func.refGene'] == ('UTR3')]
UTR3_AAC.count()

Chr              16
Start            16
End              16
Ref              16
Alt              16
                 ..
Otherinfo1224    16
Otherinfo1225    16
Otherinfo1226    16
Otherinfo1227    16
Otherinfo1228    16
Length: 1239, dtype: int64

In [106]:
# Filter UTR5 variants
UTR5_AAC = PP2A_AAC[PP2A_AAC['Func.refGene'] == ('UTR5')]
UTR5_AAC.count()

Chr              4
Start            4
End              4
Ref              4
Alt              4
                ..
Otherinfo1224    4
Otherinfo1225    4
Otherinfo1226    4
Otherinfo1227    4
Otherinfo1228    4
Length: 1239, dtype: int64

In [107]:
# Calculate freq - cases vs controls

In [108]:
reduced_coding_nonsynonymous_AAC = coding_nonsynonymous_AAC[["Chr", "Start", "End", "Gene.refGene"]]
reduced_coding_nonsynonymous_AAC.to_csv(f'{WORK_DIR}/reduced_coding_nonsynonymous_AAC.txt', sep = '\t', index = False, header= False)

In [109]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head reduced_coding_nonsynonymous_AAC.txt

9	129134882	129134882	PTPA
9	129147457	129147457	PTPA


In [110]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AAC --extract range reduced_coding_nonsynonymous_AAC.txt --assoc --out coding_nonsynonymous_pheno_PP2A_AAC --allow-no-sex --adjust

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AAC.log.
Options in effect:
  --adjust
  --allow-no-sex
  --assoc
  --bfile pheno_PP2A_AAC
  --extract range reduced_coding_nonsynonymous_AAC.txt
  --out coding_nonsynonymous_pheno_PP2A_AAC

14998 MB RAM detected; reserving 7499 MB for main workspace.
310 variants loaded from .bim file.
1216 people (474 males, 742 females) loaded from .fam.
1215 phenotype values loaded from .fam.
--extract range: 308 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1216 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%

In [111]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AAC --extract range reduced_coding_nonsynonymous_AAC.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_AAC_rare_0.05 --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_AAC_rare_0.05.log.
Options in effect:
  --allow-no-sex
  --bfile pheno_PP2A_AAC
  --extract range reduced_coding_nonsynonymous_AAC.txt
  --make-bed
  --max-maf 0.05
  --out coding_nonsynonymous_AAC_rare_0.05

14998 MB RAM detected; reserving 7499 MB for main workspace.
310 variants loaded from .bim file.
1216 people (474 males, 742 females) loaded from .fam.
1215 phenotype values loaded from .fam.
--extract range: 308 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1216 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%

In [112]:
PP2A_AAC_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AAC.assoc.adjusted', delim_whitespace=True)
PP2A_AAC_freq.head(10)

,CHR,SNP,UNADJ,GC,BONF,HOLM,SIDAK_SS,SIDAK_SD,FDR_BH,FDR_BY
0,9,chr9:129134882:A:C,0.1446,0.3442,0.2893,0.2893,0.2684,0.2684,0.2893,0.4339
1,9,chr9:129147457:C:T,0.8387,0.8950,1.0000,0.8387,0.9740,0.8387,0.8387,1.0000


In [113]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AAC --extract range reduced_coding_nonsynonymous_AAC.txt --assoc --out coding_nonsynonymous_pheno_PP2A_AAC --allow-no-sex --ci 0.95

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AAC.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile pheno_PP2A_AAC
  --ci 0.95
  --extract range reduced_coding_nonsynonymous_AAC.txt
  --out coding_nonsynonymous_pheno_PP2A_AAC

14998 MB RAM detected; reserving 7499 MB for main workspace.
310 variants loaded from .bim file.
1216 people (474 males, 742 females) loaded from .fam.
1215 phenotype values loaded from .fam.
--extract range: 308 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1216 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%

In [114]:
PP2A_AAC_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AAC.assoc', delim_whitespace=True)
PP2A_AAC_freq.head(10)

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR,SE,L95,U95
0,9,chr9:129134882:A:C,129134882,C,0.00000,0.007049,A,2.12800,0.1446,0.000,inf,0.0000,NaN
1,9,chr9:129147457:C:T,129147457,T,0.02013,0.018430,C,0.04142,0.8387,1.094,0.443,0.4593,2.607


## Calculate freq of HMZ in cases versus controls

In [115]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AAC --extract range reduced_coding_nonsynonymous_AAC.txt --recode A --out coding_nonsynonymous_pheno_PP2A_AAC

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AAC.log.
Options in effect:
  --bfile pheno_PP2A_AAC
  --extract range reduced_coding_nonsynonymous_AAC.txt
  --out coding_nonsynonymous_pheno_PP2A_AAC
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
310 variants loaded from .bim file.
1216 people (474 males, 742 females) loaded from .fam.
1215 phenotype values loaded from .fam.
--extract range: 308 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1216 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%

In [116]:
PP2A_AAC_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AAC.assoc', delim_whitespace=True)
PP2A_AAC_freq.head()

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR,SE,L95,U95
0,9,chr9:129134882:A:C,129134882,C,0.00000,0.007049,A,2.12800,0.1446,0.000,inf,0.0000,NaN
1,9,chr9:129147457:C:T,129147457,T,0.02013,0.018430,C,0.04142,0.8387,1.094,0.443,0.4593,2.607


In [117]:
PP2A_AAC_recode = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AAC.raw', delim_whitespace=True)
PP2A_AAC_recode.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
0,0,BCM_000096_s1,0,0,1,2,0.0,0.0
1,0,BCM_000142_s1,0,0,1,2,0.0,0.0
2,0,BCM_000150_s1,0,0,2,2,0.0,0.0
3,0,BCM_000173_s1,0,0,1,2,0.0,0.0
4,0,BCM_000204_s1,0,0,2,2,0.0,0.0


In [118]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile  pheno_PP2A_AAC --extract range reduced_coding_nonsynonymous_AAC.txt --recode A --out coding_nonsynonymous_pheno_PP2A_AAC

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AAC.log.
Options in effect:
  --bfile pheno_PP2A_AAC
  --extract range reduced_coding_nonsynonymous_AAC.txt
  --out coding_nonsynonymous_pheno_PP2A_AAC
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
310 variants loaded from .bim file.
1216 people (474 males, 742 females) loaded from .fam.
1215 phenotype values loaded from .fam.
--extract range: 308 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1216 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%

In [119]:
# Explore HMZs for chr9:129147457:C:T_T in cases
PP2A_hom_casos_AAC = PP2A_AAC_recode[(PP2A_AAC_recode['chr9:129147457:C:T_T'] == 2) & (PP2A_AAC_recode['PHENOTYPE'] == 2)]
PP2A_hom_casos_AAC.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T


In [120]:
PP2A_hom_casos_AAC.shape

(0, 8)

In [121]:
# Explore HMZs for chr9:129147457:C:T_T in controls
PP2A_hom_controles_AAC = PP2A_AAC_recode[(PP2A_AAC_recode['chr9:129147457:C:T_T'] == 2) & (PP2A_AAC_recode['PHENOTYPE'] == 1)]
PP2A_hom_controles_AAC.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T


In [122]:
PP2A_hom_controles_AAC.shape

(0, 8)

COMP HET

In [123]:
# Identifying individuals het for the alternative allele
het = PP2A_AAC_recode[(PP2A_AAC_recode['chr9:129147457:C:T_T'] == 1) | (PP2A_AAC_recode['chr9:129134882:A:C_C'] == 1)]
het = het.dropna()
print(het.shape)

(60, 8)


In [124]:
het.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
22,0,BIOFIND_000173_s1,0,0,2,1,0.0,1.0
30,0,BLAACPD-KPM_000016_s1,0,0,2,1,1.0,0.0
66,0,BLAACPD-UAB_000014_s1,0,0,1,2,0.0,1.0
70,0,BLAACPD-UAB_000032_s1,0,0,2,1,0.0,1.0
85,0,BLAACPD-UC_000008_s1,0,0,1,2,0.0,1.0


In [125]:
het['compound'] = het['chr9:129147457:C:T_T'] + het['chr9:129134882:A:C_C']
het['compound'].value_counts()

1.0    60
Name: compound, dtype: int64

In [126]:
comp_het_info_cases = het[(het['PHENOTYPE'] == 2) & (het['compound'] == 2)]
print(comp_het_info_cases.shape)

(0, 9)


In [127]:
comp_het_info_control = het[(het['PHENOTYPE'] == 1) & (het['compound'] == 1)]
print(comp_het_info_control.shape)

(54, 9)


In [128]:
comp_het_info_cases.to_csv('/home/jupyter/P2PA_GP2/comp_het_info_cases.txt', sep = '\t', index=True)
comp_het_info_cases.head

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/P2PA_GP2/comp_het_info_cases.txt'

In [129]:
# Explore HETs for chr9:129147457:C:T_T in cases
PP2A_het_casos_129147457_AAC = PP2A_AAC_recode[(PP2A_AAC_recode['chr9:129147457:C:T_T'] == 1) & (PP2A_AAC_recode['PHENOTYPE'] == 2)]
PP2A_het_casos_129147457_AAC

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
66,0,BLAACPD-UAB_000014_s1,0,0,1,2,0.0,1.0
85,0,BLAACPD-UC_000008_s1,0,0,1,2,0.0,1.0
147,0,CORIELL_003677_s1,0,0,2,2,0.0,1.0
164,0,CORIELL_004266_s1,0,0,2,2,0.0,1.0
166,0,CORIELL_004346_s1,0,0,2,2,0.0,1.0
1196,0,PPMI_000087_s1,0,0,2,2,0.0,1.0


In [130]:
# Explore HETs for chr9:129147457:C:T_T in controls
PP2A_het_controles_129147457_AAC = PP2A_AAC_recode[(PP2A_AAC_recode['chr9:129147457:C:T_T'] == 1) & (PP2A_AAC_recode['PHENOTYPE'] == 1)]
PP2A_het_controles_129147457_AAC

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
22,0,BIOFIND_000173_s1,0,0,2,1,0.0,1.0
70,0,BLAACPD-UAB_000032_s1,0,0,2,1,0.0,1.0
95,0,BLAACPD-UC_000021_s1,0,0,2,1,0.0,1.0
171,0,CORIELL_004672_s1,0,0,2,1,0.0,1.0
175,0,CORIELL_004696_s1,0,0,1,1,0.0,1.0
181,0,CORIELL_004736_s1,0,0,1,1,0.0,1.0
193,0,CORIELL_005187_s1,0,0,2,1,0.0,1.0
204,0,CORIELL_005874_s1,0,0,1,1,0.0,1.0
205,0,CORIELL_005948_s1,0,0,1,1,0.0,1.0
222,0,CORIELL_006457_s1,0,0,2,1,0.0,1.0


In [131]:
!pip install rpy2
%load_ext rpy2.ipython

In [132]:
import rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [145]:
PP2A_het_casos_129147457_AAC.to_csv('PP2A_het_casos_129147457_AAC.csv')

In [147]:
temp = pd.read_csv('PP2A_het_casos_129147457_AAC.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
0,66,0,BLAACPD-UAB_000014_s1,0,0,1,2,0.0,1.0
1,85,0,BLAACPD-UC_000008_s1,0,0,1,2,0.0,1.0
2,147,0,CORIELL_003677_s1,0,0,2,2,0.0,1.0
3,164,0,CORIELL_004266_s1,0,0,2,2,0.0,1.0
4,166,0,CORIELL_004346_s1,0,0,2,2,0.0,1.0
5,1196,0,PPMI_000087_s1,0,0,2,2,0.0,1.0


In [151]:
PP2A_het_casos_129147457_AAC.shape

(6, 8)

In [155]:
PP2A_het_controles_129147457_AAC.to_csv('PP2A_het_controles_129147457_AAC.csv')

In [156]:
temp = pd.read_csv('PP2A_het_controles_129147457_AAC.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
0,22,0,BIOFIND_000173_s1,0,0,2,1,0.0,1.0
1,70,0,BLAACPD-UAB_000032_s1,0,0,2,1,0.0,1.0
2,95,0,BLAACPD-UC_000021_s1,0,0,2,1,0.0,1.0
3,171,0,CORIELL_004672_s1,0,0,2,1,0.0,1.0
4,175,0,CORIELL_004696_s1,0,0,1,1,0.0,1.0
5,181,0,CORIELL_004736_s1,0,0,1,1,0.0,1.0
6,193,0,CORIELL_005187_s1,0,0,2,1,0.0,1.0
7,204,0,CORIELL_005874_s1,0,0,1,1,0.0,1.0
8,205,0,CORIELL_005948_s1,0,0,1,1,0.0,1.0
9,222,0,CORIELL_006457_s1,0,0,2,1,0.0,1.0


In [152]:
PP2A_het_controles_129147457_AAC.shape

(39, 8)

In [139]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
covs <- fread("master_key_release3_final.csv", header =T)
covs

                  GP2ID         GP2sampleID manifest_id phenotype pheno_for_qc
    1: SYNAPS-KZ_000001 SYNAPS-KZ_000001_s1          m1   Control            1
    2: SYNAPS-KZ_000002 SYNAPS-KZ_000002_s1          m1   Control            1
    3: SYNAPS-KZ_000003 SYNAPS-KZ_000003_s1          m1   Control            1
    4: SYNAPS-KZ_000004 SYNAPS-KZ_000004_s1          m1   Control            1
    5: SYNAPS-KZ_000005 SYNAPS-KZ_000005_s1          m1   Control            1
   ---                                                                        
16853:      PPMI_000789      PPMI_000789_s1          m2     Other           -9
16854:      PPMI_000781      PPMI_000781_s1          m2     Other           -9
16855:      PPMI_000818      PPMI_000818_s1          m2     Other           -9
16856:      PPMI_000841      PPMI_000841_s1          m2     Other           -9
16857:      PPMI_000808      PPMI_000808_s1          m2     Other           -9
                                  other_pheno sex_fo

data.table 1.14.6 using 2 threads (see ?getDTthreads).  Latest news: r-datatable.com
In addition: Warning message:
In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  libraries ‘/home/jupyter/packages’, ‘/usr/lib/R/site-library’ contain no packages


In [140]:
%%R
colnames(covs)[2]  <- "IID"  
covs

                  GP2ID                 IID manifest_id phenotype pheno_for_qc
    1: SYNAPS-KZ_000001 SYNAPS-KZ_000001_s1          m1   Control            1
    2: SYNAPS-KZ_000002 SYNAPS-KZ_000002_s1          m1   Control            1
    3: SYNAPS-KZ_000003 SYNAPS-KZ_000003_s1          m1   Control            1
    4: SYNAPS-KZ_000004 SYNAPS-KZ_000004_s1          m1   Control            1
    5: SYNAPS-KZ_000005 SYNAPS-KZ_000005_s1          m1   Control            1
   ---                                                                        
16853:      PPMI_000789      PPMI_000789_s1          m2     Other           -9
16854:      PPMI_000781      PPMI_000781_s1          m2     Other           -9
16855:      PPMI_000818      PPMI_000818_s1          m2     Other           -9
16856:      PPMI_000841      PPMI_000841_s1          m2     Other           -9
16857:      PPMI_000808      PPMI_000808_s1          m2     Other           -9
                                  other_pheno sex_fo

In [153]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_casos_129147457_AAC <- fread("PP2A_het_casos_129147457_AAC.csv", header =T)
PP2A_het_casos_129147457_AAC
temp <- merge(covs, PP2A_het_casos_129147457_AAC, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_casos_129147457_AAC_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_casos_129147457_AAC_extracted <- fread("PP2A_het_casos_129147457_AAC_extracted.txt", header =T)
PP2A_het_casos_129147457_AAC_extracted

                     IID              GP2ID manifest_id phenotype pheno_for_qc
1: BLAACPD-UAB_000014_s1 BLAACPD-UAB_000014          m1        PD            2
2:  BLAACPD-UC_000008_s1  BLAACPD-UC_000008          m1        PD            2
3:     CORIELL_003677_s1     CORIELL_003677          m3        PD            2
4:     CORIELL_004266_s1     CORIELL_004266          m3        PD            2
5:     CORIELL_004346_s1     CORIELL_004346          m4        PD            2
6:        PPMI_000087_s1        PPMI_000087          m1        PD            2
   other_pheno sex_for_qc  age age_of_onset age_at_diagnosis age_at_death
1:          PD          1   NA           NA               NA           NA
2:          PD          1   NA           NA               NA           NA
3:          PD          2 75.0           72               73           NA
4:          PD          2 74.0           72               73           NA
5:          PD          2 44.0           34               NA           NA
6: 

In [157]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_controles_129147457_AAC <- fread("PP2A_het_controles_129147457_AAC.csv", header =T)
PP2A_het_controles_129147457_AAC
temp <- merge(covs, PP2A_het_controles_129147457_AAC, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_controles_129147457_AAC_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_controles_129147457_AAC_extracted <- fread("PP2A_het_controles_129147457_AAC_extracted.txt", header =T)
PP2A_het_controles_129147457_AAC_extracted

                      IID              GP2ID manifest_id phenotype pheno_for_qc
 1:     BIOFIND_000173_s1     BIOFIND_000173          m1   Control            1
 2: BLAACPD-UAB_000032_s1 BLAACPD-UAB_000032          m1   Control            1
 3:  BLAACPD-UC_000021_s1  BLAACPD-UC_000021          m1   Control            1
 4:     CORIELL_004672_s1     CORIELL_004672          m6   Control            1
 5:     CORIELL_004696_s1     CORIELL_004696          m6   Control            1
 6:     CORIELL_004736_s1     CORIELL_004736          m6   Control            1
 7:     CORIELL_005187_s1     CORIELL_005187          m6   Control            1
 8:     CORIELL_005874_s1     CORIELL_005874          m6   Control            1
 9:     CORIELL_005948_s1     CORIELL_005948          m6   Control            1
10:     CORIELL_006457_s1     CORIELL_006457          m6   Control            1
11:     CORIELL_007000_s1     CORIELL_007000          m6   Control            1
12:     CORIELL_007332_s1     CORIELL_00

In [158]:
# Explore HETs for chr9:129134882:A:C_C in controls
PP2A_het_controles_129134882_AAC = PP2A_AAC_recode[(PP2A_AAC_recode['chr9:129134882:A:C_C'] == 1) & (PP2A_AAC_recode['PHENOTYPE'] == 1)]
PP2A_het_controles_129134882_AAC

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
30,0,BLAACPD-KPM_000016_s1,0,0,2,1,1.0,0.0
213,0,CORIELL_006341_s1,0,0,1,1,1.0,0.0
418,0,CORIELL_007472_s1,0,0,2,1,1.0,0.0
499,0,CORIELL_007591_s1,0,0,1,1,1.0,0.0
675,0,CORIELL_007875_s1,0,0,2,1,1.0,0.0
678,0,CORIELL_007878_s1,0,0,2,1,1.0,0.0
774,0,CORIELL_008019_s1,0,0,2,1,1.0,0.0
819,0,CORIELL_008083_s1,0,0,2,1,1.0,0.0
876,0,CORIELL_008166_s1,0,0,2,1,1.0,0.0
890,0,CORIELL_008184_s1,0,0,1,1,1.0,0.0


In [159]:
PP2A_het_controles_129134882_AAC.to_csv('PP2A_het_controles_129134882_AAC.csv')

In [160]:
temp = pd.read_csv('PP2A_het_controles_129134882_AAC.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
0,30,0,BLAACPD-KPM_000016_s1,0,0,2,1,1.0,0.0
1,213,0,CORIELL_006341_s1,0,0,1,1,1.0,0.0
2,418,0,CORIELL_007472_s1,0,0,2,1,1.0,0.0
3,499,0,CORIELL_007591_s1,0,0,1,1,1.0,0.0
4,675,0,CORIELL_007875_s1,0,0,2,1,1.0,0.0
5,678,0,CORIELL_007878_s1,0,0,2,1,1.0,0.0
6,774,0,CORIELL_008019_s1,0,0,2,1,1.0,0.0
7,819,0,CORIELL_008083_s1,0,0,2,1,1.0,0.0
8,876,0,CORIELL_008166_s1,0,0,2,1,1.0,0.0
9,890,0,CORIELL_008184_s1,0,0,1,1,1.0,0.0


In [161]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_controles_129134882_AAC <- fread("PP2A_het_controles_129134882_AAC.csv", header =T)
PP2A_het_controles_129134882_AAC
temp <- merge(covs, PP2A_het_controles_129134882_AAC, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_controles_129134882_AAC_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_controles_129134882_AAC_extracted <- fread("PP2A_het_controles_129134882_AAC_extracted.txt", header =T)
PP2A_het_controles_129134882_AAC_extracted

                      IID              GP2ID manifest_id phenotype pheno_for_qc
 1: BLAACPD-KPM_000016_s1 BLAACPD-KPM_000016          m1   Control            1
 2:     CORIELL_006341_s1     CORIELL_006341          m6   Control            1
 3:     CORIELL_007472_s1     CORIELL_007472          m6   Control            1
 4:     CORIELL_007591_s1     CORIELL_007591          m6   Control            1
 5:     CORIELL_007875_s1     CORIELL_007875          m6   Control            1
 6:     CORIELL_007878_s1     CORIELL_007878          m6   Control            1
 7:     CORIELL_008019_s1     CORIELL_008019          m6   Control            1
 8:     CORIELL_008083_s1     CORIELL_008083          m6   Control            1
 9:     CORIELL_008166_s1     CORIELL_008166          m6   Control            1
10:     CORIELL_008184_s1     CORIELL_008184          m6   Control            1
11:     CORIELL_008211_s1     CORIELL_008211          m6   Control            1
12:     CORIELL_008335_s1     CORIELL_00

## Save out results..!

In [163]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR} {WORKSPACE_BUCKET}')

Executing: gsutil -mu terra-02e7cd23 cp -r /home/jupyter/PP2A_GP2/ gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0


Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A.annovar.hg38_multianno.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/reduced_coding_nonsynonymous.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/PP2A_het_controles_129147457_EUR_extracted.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A.vcf [Content-Type=text/vcard]...
Copying file:///home/jupyter/PP2A_GP2/coding_nonsynonymous_AJ_rare_0.05.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/reduced_coding_nonsynonymous_AFR.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/coding_nonsynonymous_pheno_PP2A_AAC.raw [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_CAS.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_EAS.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_SAS.vc